In [1]:
import pymongo
import time
import random
import string
import re
import os
import gensim



In [37]:

MongoClient = pymongo.MongoClient
conn = MongoClient('mongo-enron', 27017)
#use enron database
db = conn['enron']
messages = db.messages
messageswords = db.messageswords
topics = db.topics
authortopics = db.authortopics


def generate_topics(model,num_topics,num_words):
    #build a collection of topics based on the model
    db_records = get_topic_words(model,num_topics,num_words)
    #print db_records
    topics.drop()
    object_ids = topics.insert_many(db_records, ordered=False).inserted_ids

    return True

def get_topic_words(model,num_topics,num_words):
    rtn = []
    i=0
    for topic in model.show_topics(num_topics,num_words):
        topic_dict = dict()
        topic_dict['name'] = "topic%s" % i
        topic_dict['nodes'] = []
        for word, probability in model.show_topic(topic[0]):
            node = dict()
            node['name'] = word
            node['value'] = probability
            topic_dict['nodes'].append(node)
        i+=1
        rtn.append(topic_dict)
    return rtn
              
def print_topics(model,num_topics,num_words):
    i=0
    for topic in model.show_topics(num_topics,num_words):
        words = ''
        for word, probability in model.show_topic(topic[0]):
            words += word + ' '
        i+=1
        print("Topic %s Words: %s" % (i, words))

        

def get_authors():
    rtn = dict()
    pipeline=[
        {"$group":{"_id":"$from"}}
    ]
    rtn = list(messageswords.aggregate(pipeline, allowDiskUse=True))
    return rtn

def generate_author_topics(model):
    author_topics = []
    r = re.compile('<.*')
    for author in get_authors():
        record = dict()
        record['from'] = author['_id']
        try:
            for topictuple in model.get_author_topics(author['_id']):
                record['topic'+str(topictuple[0])] = topictuple[1]
            author_topics.append(record)
        except:
            pass
    print author_topics
    
    authortopics.drop()
    object_ids = authortopics.insert_many(author_topics, ordered=False).inserted_ids
    
    return object_ids


def main():
    start_time = time.time()
    dictionary_file = '../../gensim/at-working/dictionary'
    model_file = '../../gensim/at-working/model'
    num_topics=10
    num_words=10
    
    if not os.path.isfile(dictionary_file) or not os.path.isfile(model_file):
        return

    dictionary = gensim.corpora.Dictionary.load(dictionary_file)
    model = gensim.models.AuthorTopicModel.load(model_file)

    #generate_topics(model,num_topics,num_words)

    #print_topics(model,num_topics,num_words)

    generate_author_topics(model)

    print("---done %s seconds ---" % (time.time() - start_time))
    
main()


[{'topic9': 0.080954142779132071, 'topic8': 0.018215818320562436, 'from': u'enron.announcements@enron.com', 'topic1': 0.68227009118470949, 'topic0': 0.012319617187926294, 'topic3': 0.078410495211258716, 'topic2': 0.01543372233857602, 'topic5': 0.010024049246782444, 'topic7': 0.011554136450941456, 'topic6': 0.090817033968826302}, {'topic9': 0.058057124504112288, 'topic0': 0.065065411820959443, 'topic3': 0.21408447042830822, 'from': u'darrell.schoolcraft@enron.com', 'topic7': 0.65324784597025787}, {'topic1': 0.11011131668129956, 'topic9': 0.28565666076228563, 'topic3': 0.56227431321512844, 'from': u'sally.beck@enron.com', 'topic6': 0.024662521963742544}, {'topic9': 0.30270024172223203, 'topic3': 0.63279885422267601, 'from': u'barry.tycholiz@enron.com', 'topic7': 0.053781186905679464, 'topic6': 0.010671231516371723}, {'topic1': 0.044527731590017175, 'topic9': 0.11565859892910835, 'topic3': 0.33969261054612104, 'from': u'audrey.robertson@enron.com', 'topic7': 0.49922831313669097}, {'topic1